In [696]:
# Extrahiere Tabelle / Philosophennamen aus:
# https://de.wikipedia.org/wiki/Zeittafel_zur_Philosophiegeschichte
#
# Mozilla Firefox shortcuts
# 
# 
# Jupyter Shortcuts: 'H'
#
# https://www.tutorialspoint.com/python_web_scraping/python_modules_for_web_scraping.htm
# https://www.freecodecamp.org/news/how-to-scrape-websites-with-python-2/
# https://oxylabs.io/blog/python-web-scraping
# https://www.freecodecamp.org/news/web-scraping-python-tutorial-how-to-scrape-data-from-a-website/
# https://www.dataquest.io/blog/web-scraping-python-using-beautiful-soup/
#
# https://www.geeksforgeeks.org/parsing-tables-and-xml-with-beautifulsoup/
# https://towardsdatascience.com/a-guide-to-scraping-html-tables-with-pandas-and-beautifulsoup-7fc24c331cf7
#

In [697]:
import sys
import re
import regex
import requests
import csv
import locale
from bs4 import BeautifulSoup
# sys.setdefaultencoding('UTF8')


In [698]:
web_page_address = "https://de.wikipedia.org/wiki/Zeittafel_zur_Philosophiegeschichte"
print (web_page_address)

https://de.wikipedia.org/wiki/Zeittafel_zur_Philosophiegeschichte


In [699]:
web_page = requests.get(web_page_address)
print ("web_page = \t\t", web_page)
print ("web_page.status_code = \t", web_page.status_code)


web_page = 		 <Response [200]>
web_page.status_code = 	 200


In [700]:
# print ("web_page.content = \t", web_page.content)
# from >timeline.xsd<
# >add_*< keys are used internally
event_fieldnames = ("start", "end", "text", "progress", "fuzzy", "locked", "ends_today", "category", "description", "hyperlink", "alert", "icon", "default_color", "milestone")

additional_event_fieldnames = ("add_start_vCh", "add_end_vCh", "add_start_end_ok")

extended_event_fieldnames = []
for item in event_fieldnames:
    extended_event_fieldnames.append(item)
for item in additional_event_fieldnames:
    extended_event_fieldnames.append(item)
extended_event_fieldnames = tuple(extended_event_fieldnames)

# https://stackoverflow.com/questions/16730339/python-add-item-to-the-tuple
def csv_event_file_header_get():
    event_header = []
    for item in event_fieldnames:
        event_header.append(item)
    event_header = tuple(event_header)
    return event_header


In [701]:
soup = BeautifulSoup(web_page.content, 'html.parser')
# print(soup.prettify())

In [702]:
bs4_table = soup.find('table', class_='wikitable toptextcells')
# print (type(bs4_table))

In [703]:
def test_write_csv_with_header_file():
    fn = 'csv_tst.csv'
    # with open(sys.argv[1], 'wt') as f:
    with open(fn, 'wt') as f:
        writer = csv.writer(f)
        writer.writerow(csv_event_file_header_get())
        for i in range(3):
            row = (
                i + 1,
                chr(ord('a') + i),
                '08/{:02d}/07'.format(i + 1),
                i,
            )
            writer.writerow(row )

    # print(open(sys.argv[1], 'rt').read())    pass
    print (' -------------------')
    print(open(fn, 'rt').read())

In [704]:
test_write_csv_with_header_file()

 -------------------
start,end,text,progress,fuzzy,locked,ends_today,category,description,hyperlink,alert,icon,default_color,milestone

1,a,08/01/07,0

2,b,08/02/07,1

3,c,08/03/07,2




In [705]:
def csv_event_make():
    pass

In [706]:
def new_d_event_philosopher():
    # return dictionary with keys according to event_fieldnames in >timeline.xsd<
    # all values are preset = ''
    d_event_philosopher = {}
    for cnt, event_fieldname in enumerate (extended_event_fieldnames):
        d_event_philosopher[event_fieldname] = ''
    # d_event_philosopher["start"] = '*'
    # d_event_philosopher["end"] = '*'
    return d_event_philosopher

In [707]:
def adjust_philosopher_name(td_item):
    philosopher_name = '?'
    if td_item.a:
        philosoph_full_name = td_item.a.string                 # Name
        if ( ' von ' in philosoph_full_name) or  ( ' der ' in philosoph_full_name):
            philosopher_name = philosoph_full_name
        else:
            philosopher_name = philosoph_full_name.split(' ')[-1]
            if philosoph_full_name.split(' ')[:-1]:
                philosopher_name = philosopher_name + ', '
            for ele in philosoph_full_name.split(' ')[:-1]:
                philosopher_name = philosopher_name + ele + ' '
    return philosopher_name

In [708]:
def check_lifespan_era(philosoph_lifespan, d_event_philosopher):
    # eruiert ob vor oder nach Christus
    # setzt entsprechende Felder im dict >d_event_philosopher<

    vor_Chr  = re.search('v\. Chr\.', philosoph_lifespan)
    nach_Chr = re.search('n\. Chr\.', philosoph_lifespan)

    if vor_Chr and nach_Chr:
        d_event_philosopher['add_start_vCh'] = 'True'
        d_event_philosopher['add_end_vCh']   = 'False'
        # print ('vor_Chr and nach_Chr')
    elif (len(re.findall('v\. Chr\.', philosoph_lifespan)) >= 1):
        d_event_philosopher['add_start_vCh'] = 'True'
        d_event_philosopher['add_end_vCh']   = 'True'
    return d_event_philosopher

In [709]:
def modify_lifespan(substring, philosoph_lifespan, d_event_philosopher, comment):
    # corrects string by deleting >substring< in string >philosoph_lifespan<
    # ... if substring is present. (i.e. >n. Chr.<, >Jh.<, etc)
    # Usually this means, that lifespan is not exactly known
    # - a correspondig flag ('fuzzy') is set.
    # - a 'comment' string is returned, to indicate the change.
    found = re.search(substring, philosoph_lifespan)
    if found:
        comment = substring
        d_event_philosopher['fuzzy'] = 'True'
        philosoph_lifespan = re.sub(substring, "", philosoph_lifespan).strip()
    return philosoph_lifespan, d_event_philosopher, comment

In [717]:
cnt = 1
def adjust_philosoph_lifespan (philosoph_lifespan, d_event_philosopher):
    global cnt

    d_event_philosopher['start'] = '?'
    d_event_philosopher['end']   = '?'
    d_event_philosopher['fuzzy'] = ''
    comment = ''

    philosoph_lifespan = philosoph_lifespan.strip()
    p_l_span_org       = philosoph_lifespan

    check_lifespan_era(philosoph_lifespan, d_event_philosopher)

    loc_substring = u"um"
    philosoph_lifespan, d_event_philosopher, comment = \
        modify_lifespan(loc_substring, philosoph_lifespan, d_event_philosopher, comment)
    if comment == loc_substring:
        comment = u"ok"

    loc_substring = u"† nach"
    philosoph_lifespan, d_event_philosopher, comment = \
        modify_lifespan(loc_substring, philosoph_lifespan, d_event_philosopher, comment)
    if comment == loc_substring:
        if regex.match(u"\d{1,4}", philosoph_lifespan):
            years = regex.findall(u"\d{1,4}", philosoph_lifespan)
            d_event_philosopher['end']   = years[0]
            comment = u"ok"

    loc_substring = u"†"
    philosoph_lifespan, d_event_philosopher, comment = \
        modify_lifespan(loc_substring, philosoph_lifespan, d_event_philosopher, comment)
    if comment == loc_substring:
        if regex.match(u"\d{1,4}", philosoph_lifespan):
            years = regex.findall(u"\d{1,4}", philosoph_lifespan)
            d_event_philosopher['end']   = years[0]
            comment = u"ok"

    loc_substring = u"unsicher"
    philosoph_lifespan, d_event_philosopher, comment = \
        modify_lifespan(loc_substring, philosoph_lifespan, d_event_philosopher, comment)

    loc_substring = u"\*"
    philosoph_lifespan, d_event_philosopher, comment = \
        modify_lifespan(loc_substring, philosoph_lifespan, d_event_philosopher, comment)
    if comment == loc_substring:
        if regex.match(u"\d{1,4}", philosoph_lifespan):
            years = regex.findall(u"\d{1,4}", philosoph_lifespan)
            d_event_philosopher['start']   = years[0]
            comment = u"ok"

    # loc_substring = u"v\. Chr\."
    # philosoph_lifespan, d_event_philosopher, comment = \
    #     modify_lifespan(loc_substring, philosoph_lifespan, d_event_philosopher, comment)

    loc_substring = u"Jh\."
    philosoph_lifespan, d_event_philosopher, comment = \
        modify_lifespan(loc_substring, philosoph_lifespan, d_event_philosopher, comment)
    if (comment == loc_substring) and regex.match(u"\d{1,4}", philosoph_lifespan):
        years = regex.findall(u"\d{1,4}", philosoph_lifespan)
        try:     d_event_philosopher['end'] = str(int(years[0]) * 100)
        except:  d_event_philosopher['end'] = '??'
        comment = u"ok"

    loc_substring = u"Jahrhundert"
    philosoph_lifespan, d_event_philosopher, comment = \
        modify_lifespan(loc_substring, philosoph_lifespan, d_event_philosopher, comment)
    if (comment == loc_substring) and regex.match(u"\d{1,4}", philosoph_lifespan):
        years = regex.findall(u"\d{1,4}", philosoph_lifespan)
        try:    d_event_philosopher['end'] = str(int(years[0]) * 100)
        except: d_event_philosopher['end'] = '??'
        comment = u"ok"

    if regex.match(u"\d{1,4}–\d{1,4}$", philosoph_lifespan):
        years = regex.findall(u"\d{1,4}", philosoph_lifespan)
        d_event_philosopher['start'] = years[0]
        d_event_philosopher['end']   = years[1]
        comment = u"\d{1,4}–\d{1,4}$"
        comment = u"ok"
    elif regex.match(u"d{1,4}$", philosoph_lifespan):
        years = regex.findall(u"\d{1,4}", philosoph_lifespan)
        d_event_philosopher['start'] = years[0]
        d_event_philosopher['end']   = years[0]
        comment = u"ok"
        # print (philosoph_lifespan, ' >>>> ', d_event_philosopher['start'] , d_event_philosopher['end'])
        pass

    p_l_span = philosoph_lifespan
    # p_l_span_org
    if (comment != u"ok") or (d_event_philosopher['add_start_vCh']) or (d_event_philosopher['add_end_vCh']) or  (d_event_philosopher['fuzzy']):
        print ('{:d}'.format(cnt), '{:25}'.format(p_l_span_org), '=>', '{:6}'.format(d_event_philosopher['start']) , '{:6}'.format(d_event_philosopher['end']), ' | ', '{:6}'.format(d_event_philosopher['add_start_vCh']), '{:6}'.format(d_event_philosopher['add_end_vCh']), comment)
        cnt = cnt + 1
    return philosoph_lifespan, d_event_philosopher['fuzzy']

In [718]:
row_cnt = 0

act_h2  = ''   # Epoche
act_h3  = ''   # Unterepoche
act_h4  = ''   # Unter-Unterepoche

old_h2  = ''
old_h3  = ''
old_h4  = ''


# lo == list of : event == philosopher
lo_d_event_philosopher  = []
d_event_philosopher     = new_d_event_philosopher()
d_event_philosoph_cnt   = 0

# cycle rows in table:
for bs4_row in bs4_table.find_all('tr'):
    # print (type(bs4_row))   # type == <class 'bs4.element.Tag'>
    bs4_row_text = BeautifulSoup(bs4_row.get_text())  # get_text() == 'converts' type to string, sort of
    row_cnt += 1
    # https://stackoverflow.com/questions/13853025/is-there-a-way-in-beautiful-soup-to-count-the-number-of-tags-in-a-html-page
    td_columns = len (bs4_row.find_all('td'))

    # 1 column in row == Epoche / Unterepoche / Unter-Unterepoche == Headline of span
    if (td_columns == 1):
        if bs4_row.find_all('h2'):
            try:
                act_h2 = bs4_row.find_all('span', class_='mw-headline')[0].string
                category = act_h2
            except:  pass
        elif bs4_row.find_all('h3'):
            try:
                act_h3 = bs4_row.find_all('span', class_='mw-headline')[0].string
                category = act_h2 + ' - ' + act_h3
            except: pass
        elif bs4_row.find_all('h4'):
            try:
                act_h4 = bs4_row.find_all('span', class_='mw-headline')[0].string
                category = act_h2  + ' - ' + act_h3  + ' - ' + act_h4
            except: pass
        # print ('category =  ', category)

    elif (td_columns == 4) :
        d_event_philosopher     = new_d_event_philosopher()
        philosoph_lifespan      = None

        # cycle 4 columns in row:  'Periode' / 'Philosoph' (Name) / 'Philosophie' (Thesen) / 'Allgemeine Geschichte'
        for td_cnt, td_item in enumerate (bs4_row.find_all('td')):
            if   (td_cnt == 0) and (td_item.string):                               #  'Periode'
                philosoph_lifespan = adjust_philosoph_lifespan (td_item.string, d_event_philosopher)
            elif (td_cnt == 1) and (td_item.a.string):                             #  'Philosoph' (Name)
                philosopher_name = adjust_philosopher_name(td_item)
                d_event_philosoph_cnt += 1
                try:
                    philosoph_lifespan = philosoph_lifespan.string.replace("\n", " ")
                except:
                    philosoph_lifespan = '???? '
            elif (td_cnt == 2) and (td_item):                                      #  'Philosophie' (Thesen)
                philosopher_lo_theories = []
                for li_cnt, li_item in enumerate (td_item.find_all('li')):
                    philosopher_lo_theories.append(li_item.text)
            elif (td_cnt == 3) and (td_item):                                      #  'Allgemeine Geschichte'
                # philosopher_lo_history = td_item.text
                philosopher_lo_history = []
                for li_cnt, li_item in enumerate (td_item.find_all('li')):
                    philosopher_lo_history.append(li_item.text)

                line_new = '{:>35}      {}    {}  '.format(philosoph_lifespan, philosopher_name, philosopher_lo_theories)
                # print (d_event_philosoph_cnt, ': ', line_new)

                # philosopher.lifespan = philosoph_lifespan


    # lo_d_event_philosopher.append(philosopher)



1 um 624–546                => 624    546     |                ok
2 um 610–547                => 610    547     |                ok
3 um 585–525                => 585    525     |                ok
4 um 499–428                => 499    428     |                ok
5 um 570–470                => 570    470     |                ok
6 um 515–445                => 515    445     |                ok
7 um 490–430                => 490    430     |                ok
8 um 490–430                => 490    430     |                ok
9 um 540–480                => 540    480     |                ok
10 um 499–428                => 499    428     |                ok
11 um 494–434                => 494    434     |                ok
12 um 580–500                => 580    500     |                ok
13 um 500                    => ?      ?       |                ok
14 um 500                    => ?      ?       |                ok
15 um 470–399                => 470    399     |                ok
16 u